In [1]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()

In [23]:
"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(191,300):
    if len(gap.IdsOfAllSmallGroups(i+1)) < 150:
        small += gap.IdsOfAllSmallGroups(i+1)
    else:
        print(i+1)
# small += gap.IdsOfAllSmallGroups(96)

In [ ]:
"""For original problem with only trivial character."""
class induce():
    """Function to return list of class sizes in default order."""
    def __init__(self):
        pass

    """Returns class sizes up to conjugacy"""
    def class_sizes(self, G):
        group_order = G.order()
        reps = G.conjugacy_classes_representatives()
        class_sizes = []
        for g in reps:
            class_sizes.append(group_order / G.centralizer(g).order())
#         print(class_sizes)
        return class_sizes

    """Function to find all subgroups. Returns list of subgroups."""
    def subgroups(self, G):
        subgroups = G.conjugacy_classes_subgroups()
#         print(subgroups)
        return subgroups

    """Function to find induced trivial character of each subgroup. Returns a list of lists."""
    def get_induced(self, subgroups, G):
        ind = []
        for subgroup in subgroups:
            ind.append(subgroup.trivial_character().induct(G))
#         print(ind)
        return ind

    """Forms the matrices that we use to solve the simultaneous equation later. 
    matrix[1:-1]: Excludes the first and last subgroup (trivial subgroup and the group itself), and the first column.
    checkmat[1:-1] Used for checking value at identity"""
    def mat(self, induced):
        matrix = []
        for char in induced:
            matrix.append(list(char.values()))
        neg = []
        for i in range(len(matrix[0])):
            neg.append(-1)
        matrix[-1] = neg
#         for line in matrix:
#             print(line)
        return matrix[1:]

    """Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise."""
    def test(self, G):
        a = self.class_sizes(G)
        b = self.subgroups(G)
        c = self.get_induced(b, G)
        d = self.mat(c)
        e = Matrix(d).T
        temp = []
        temp.append(G.order())
        for i in range(len(a)-1):
            temp.append(0)
        Y = vector(temp)
        p = MixedIntegerLinearProgram(solver='GLPK')
        w = p.new_variable(nonnegative=True)
        p.add_constraint(e*w == Y)
        try:
            p.solve()
#             out = []
#             for i, v in (p.get_values(w).items()):
#                 out.append('w_%s = %s' % (i, v))
#                 """Prints subgroup and subgroup order."""
#                 if round(v,6) != 0:
#                     print(b[i+1].order())
            """Prints the solution and SmallGroup Id"""
#             print(out)
            print(str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]))
        except:
            pass
#             print('fail')

In [6]:
"""Main function
If group is in id form.
https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
This works on CoCalc/non-windows system with gap_packages library installed."""
if __name__ == "__main__":
    inducer = induce()
    groups = small
    for groupid in groups:
        G = gap.SmallGroup(groupid)
        try:
            inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
        except:
            pass
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
#     D1 = CyclicPermutationGroup(5)
#     D2 = CyclicPermutationGroup(4)
#     groups = [direct_product_permgroups([D1,D1,D1])]
#     for group in groups:
#         try:
#             inducer.test(group)
#         except:
#             pass

2.0
[ 12, 4 ]
